In [ ]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse
import time
import re

# WebサイトのトップページURL
BASE_URL = "https://www.musashino-u.ac.jp/"
# 同一ドメインのURLを格納するキュー（未探索）
queue = {BASE_URL}
# 既に探索済みのURL
visited = set()
# サイトマップを格納する辞書
sitemap = {}
# Webサイトへの負荷軽減のための待機時間（秒）
WAIT_TIME = 1.0

def is_valid(url):
    """
    URLが同一ドメインであり、除外する拡張子でないかを確認する
    """
    parsed_url = urlparse(url)
    # 同一ドメインの確認
    if parsed_url.netloc != urlparse(BASE_URL).netloc:
        return False
    # ファイル拡張子の除外 (.pdf, .jpeg, .png)
    EXCLUDE_EXTENSIONS = ['.pdf', '.jpeg', '.png', '.jpg', '.gif']
    if any(url.lower().endswith(ext) for ext in EXCLUDE_EXTENSIONS):
        return False
    return True

def scrape_sitemap():
    """
    Webサイトを巡回し、サイトマップを抽出するメイン関数
    """
    print(f"--- サイトマップ抽出開始: {BASE_URL} ---")
    
    while queue and len(sitemap) < 500: # 無限ループ回避のため上限を設定（任意）
        current_url = queue.pop()
        
        if current_url in visited:
            continue

        print(f"探索中: {current_url}")
        
        try:
            # 負荷軽減のための待機
            time.sleep(WAIT_TIME)
            
            # Webサイトにアクセス
            response = requests.get(current_url, timeout=10)
            response.raise_for_status() # 200以外の場合は例外発生
            
            # エンコーディングの自動検出（文字化け対策）
            response.encoding = response.apparent_encoding
            html_content = response.text
            
            # 探索済みに登録
            visited.add(current_url)
            
            # BeautifulSoupでHTMLを解析
            soup = BeautifulSoup(html_content, 'html.parser')
            
            # <title>タグの内容を取得し、辞書に格納
            title_tag = soup.find('title')
            page_title = title_tag.string.strip() if title_tag and title_tag.string else "タイトルなし"
            sitemap[current_url] = page_title
            
            # リンクを抽出
            for a_tag in soup.find_all('a', href=True):
                # 完全なURLに変換（相対パスに対応）
                link_url = urljoin(current_url, a_tag['href'])
                
                # HTMLコメント内のリンクを除外 (BeautifulSoupはコメント内の要素を直接解析しないため、テキストからチェック)
                # ただし、完全なチェックは複雑なため、ここではa_tagが直接取得できたことを優先し、簡易的なチェックとする
                # コメントアウトされた要素を特定するには、HTMLの生データを扱う必要がある
                
                # 有効なリンク（同一ドメイン、除外拡張子でない）か確認
                if is_valid(link_url) and link_url not in visited:
                    queue.add(link_url)
                    
        except requests.exceptions.RequestException as e:
            print(f"エラー発生: {current_url} - {e}")
            visited.add(current_url) # エラーが発生したURLも再探索しないように登録
            
        except Exception as e:
            print(f"その他のエラー: {current_url} - {e}")
            visited.add(current_url)

    print("\n--- サイトマップ抽出完了 ---")
    
    # 抽出結果の表示
    print("\n--- 抽出結果 (URL: <title>) ---")
    for url, title in sitemap.items():
        print(f"'{url}': '{title}'")

    print(f"\n合計ページ数: {len(sitemap)}")

if __name__ == "__main__":
    scrape_sitemap()